## Import Library

In [1]:
import logging
from typing import Dict, Tuple
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
import random
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, roc_curve, precision_score
from sklearn.model_selection import train_test_split
from kedro.extras.datasets.pickle import PickleDataSet
from sklearn.model_selection import GridSearchCV
import xmltodict

## Load/Save Data (XML/CSV)

In [2]:
def _clean_name (cadena):
    
    cadena_str= str(cadena)
    cadena_str=cadena_str[26:]
    cadena_str = cadena_str.replace('}', '',1)
    cadena_str = cadena_str.replace('\'','')
    
    return cadena_str

In [3]:
def load_from_xml (path):
    xml=open(path, encoding='ISO-8859-1')
    xmldict = xmltodict.parse(xml.read())
    df_enfermedades = pd.DataFrame()
    lista_enfer_sinto_prob=[]
    first_tree=xmldict["JDBOR"]["HPODisorderSetStatusList"]["HPODisorderSetStatus"]
    id=0
    for nodo in xmldict["JDBOR"]["HPODisorderSetStatusList"]["HPODisorderSetStatus"]:
     
        enfermedad=_clean_name(nodo["Disorder"]["Name"])
        sec_tree= nodo["Disorder"]["HPODisorderAssociationList"]
        if (len(sec_tree)==2):
            tam_sintomas=len(sec_tree["HPODisorderAssociation"])
            i=0
            registro_enfer_sinto_prob=[]
            while (i<tam_sintomas):
                registro_enfer_sinto_prob.append(enfermedad)
                registro_enfer_sinto_prob.append(id)
                registro_enfer_sinto_prob.append(sec_tree["HPODisorderAssociation"][i]["HPO"]["HPOTerm"])
                frecuencia=_clean_name(sec_tree["HPODisorderAssociation"][i]["HPOFrequency"]["Name"])
                registro_enfer_sinto_prob.append(frecuencia)
                lista_enfer_sinto_prob.append (registro_enfer_sinto_prob)
                registro_enfer_sinto_prob=[]
                i=i+1
        id=id+1
    df_enfermedades = pd.DataFrame(lista_enfer_sinto_prob)
    df_enfermedades = df_enfermedades.rename(columns={0:'Enfermedad',1:'Id_Enfermedad', 2:'Sintoma', 3:"Frecuencia"})
    
    return df_enfermedades        

In [4]:
def save_to_csv (data, path):
    data.to_csv(path, index=False)

In [5]:
def load_from_csv (path):
    data=pd.read_csv(path)
    return data   

## Exploratory Data Analysis (EDA)

In [6]:
def clean_data(data): 
    
    
    data=data.drop_duplicates()
    data=data.dropna()
    vc = data["Sintoma"].value_counts()
    vector=vc[vc < 100].index
    for a in vector:
        indexNames = data [ data["Sintoma"] == a ].index
        for b in indexNames:
            data.drop(b , inplace=True, axis=0)
  
    return data 

## Feature Engineering 

In [7]:
def selection_and_preparation_data(data):
    
    #data=data[(data['Frecuencia']=="Muy frecuente (99-80%)")]
    data=data[(data['Frecuencia']=="Muy frecuente (99-80%)") | (data['Frecuencia']=="Frecuente (79-30%)")]
    #data=data[(data['Frecuencia']=="Frecuente (79-30%)")]

        
    return data


In [8]:
 data=data[(data['Frecuencia']=="Muy frecuente (99-80%)") | (data['Frecuencia']=="Frecuente (79-30%)")]

NameError: name 'data' is not defined

In [8]:
def generate_data_train (data, repeticiones):
    
    sintomas=data.iloc[:,1]
    sintomas_sin_repe=sintomas.drop_duplicates()
    
    df_train=pd.DataFrame(columns=sintomas_sin_repe)
   # df_train.insert(0, 'Enfermedad', 0)
    df_train.insert(0, 'id_Enfermedad', 0)
 
    
    data_agrupado = (data.groupby("Enfermedad")
         .agg({"Sintoma": np.array, "Frecuencia": np.array})
         .reset_index()
         )
    
    z=0
    j=0
    while (z<repeticiones):
    
  #  print ("entra")
        i=0
        for a in data_agrupado["Enfermedad"]:
           # print ("Enfermedad: ", a)
        #vector_enfermedad.append(a)
        #lista=[]
            lst = [0] * ((len(sintomas_sin_repe)+1))
           
            df_train.loc[len(df_train)] = lst
           # df_train["Enfermedad"][j]=a
            df_train["id_Enfermedad"][j]=(i+1)
           
            pos=0
            for b in data_agrupado["Sintoma"][i]:
                valor_aleatorio = random.random()
                frecuencia=data_agrupado["Frecuencia"][i][pos]
            #    print ("frecuencia:", frecuencia)
                if (frecuencia=="Muy frecuente (99-80%)"):
             #       print (b)
              #      print ("es muy frecuente")
                    if (valor_aleatorio>0.4):
                        valor_entero=1
                    else:
                        valor_entero=0
                elif (frecuencia=="Frecuente (79-30%)"):
               #     print (b)
                #    print ("es frecuente")
                    if (valor_aleatorio>0.6):
                        valor_entero=1
                    else:
                        valor_entero=0
                df_train[b][j]=valor_entero
               
                pos=pos+1
            j=j+1
            i=i+1
        z=z+1
        print ("VUELTA: ", z)
        print ("Tamaño: ",j)
    return df_train        

## Train Data and Predict

In [9]:
def train_data (df_train):
    
    target=df_train["id_Enfermedad"]
   # data=data.drop (['Enfermedad'], axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(df_train,target)
    rf= RandomForestClassifier()
    rf.fit (X_train,y_train)
    #prediccion=rf.predict(X_test)
    
    return df_train,rf,target,X_test,y_test, X_train, y_train


In [10]:
#def rank (df_train,rf,X_test):
  #  predicciones = rf.predict_proba(X_test)
   # predicciones[:5, :]
   # df_rank = pd.DataFrame(data=predicciones, columns=['0', '1','2','3','4', '5', '6', '7', '8'])
   # df_rank['clasificacion_default_0.5'] = np.where(df_predicciones['0'] > df_predicciones['1'], 0, 1)
   # return df_rank

## Evaluate

In [29]:
def evaluate (classifier, X_test, y_test):

    y_pred=classifier.predict(X_test)
    probs=classifier.predict_proba(X_test) [:, 1]
    
    acc = accuracy_score(y_test, y_pred)
    pres= precision_score (y_test, y_pred, average='micro')
    f1 = f1_score(y_test, y_pred, average='micro')
    
   # roc_auc = roc_auc_score(y_test,probs, multi_class='ovr')
    #auc = roc_auc_score(y_test, probs)
    print ("Accurancy: ", acc)
    print ("Precision: ", pres)
    print ("f1: ", f1)
    #print ("auc: ", auc)
    
   # fpr, tpr, threshols = roc_curve(y_test, probs)
   # plt.plot(fpr,tpr)
   # plt.xlabel('False positives rate')
   # plt.ylabel('True positives rate')
   # plt.gcf().set_size_inches(8,6)
   

## Notebook Main

In [12]:
df_enfermedades=load_from_xml("../data/01_raw/enfermedades.xml")

In [13]:
df_enfermedades[df_enfermedades["Enfermedad"]=="Acalasia idiopática"]

,Enfermedad,Id_Enfermedad,Sintoma,Frecuencia
75498,Acalasia idiopática,3276,Dysphagia,Muy frecuente (99-80%)
75499,Acalasia idiopática,3276,Weight loss,Frecuente (79-30%)
75500,Acalasia idiopática,3276,Gastroesophageal reflux,Frecuente (79-30%)
75501,Acalasia idiopática,3276,Bronchitis,Frecuente (79-30%)
75502,Acalasia idiopática,3276,Cough,Frecuente (79-30%)
75503,Acalasia idiopática,3276,Chest pain,Frecuente (79-30%)
75504,Acalasia idiopática,3276,Recurrent aspiration pneumonia,Ocasional (29-5%)
75505,Acalasia idiopática,3276,Malnutrition,Ocasional (29-5%)
75506,Acalasia idiopática,3276,Wheezing,Ocasional (29-5%)
75507,Acalasia idiopática,3276,Decreased prealbumin level,Ocasional (29-5%)


In [14]:
save_to_csv(df_enfermedades,"../data/01_raw/exportCSVfromXML_Ofi.csv")

In [9]:
df_enfermedades= load_from_csv("../data/01_raw/exportCSVfromXML_Ofi.csv")

In [10]:

    print ("Enfermedades: ", df_enfermedades["Enfermedad"].nunique())
    print ("Sintomas: ", df_enfermedades["Sintoma"].nunique())
    print ("Frecuencias: ", df_enfermedades["Frecuencia"].nunique())

Enfermedades:  4262
Sintomas:  8303
Frecuencias:  6


In [13]:
df_enfermedades_cleaned=clean_data(df_enfermedades)

In [ ]:
  data_agrupado = (data.groupby("Enfermedad")
         .agg({"Sintoma": np.array, "Frecuencia": np.array})
         .reset_index()
         )

In [17]:
  print ("Después de borrado filas, duplicates y NAN")
  print ("Enfermedades: ", df_enfermedades_cleaned["Enfermedad"].nunique())
  print ("Sintomas: ", df_enfermedades_cleaned["Sintoma"].nunique())
  print ("Frecuencias: ", df_enfermedades_cleaned["Frecuencia"].nunique())  


Después de borrado filas, duplicates y NAN
Enfermedades:  4014
Sintomas:  196
Frecuencias:  6


In [18]:
df_enfermedades_cleaned.head()

,Enfermedad,Id_Enfermedad,Sintoma,Frecuencia
0,Enfermedad de Alexander,0,Macrocephaly,Muy frecuente (99-80%)
1,Enfermedad de Alexander,0,Intellectual disability,Muy frecuente (99-80%)
2,Enfermedad de Alexander,0,Seizure,Muy frecuente (99-80%)
3,Enfermedad de Alexander,0,Spasticity,Muy frecuente (99-80%)
4,Enfermedad de Alexander,0,Agenesis of corpus callosum,Muy frecuente (99-80%)


In [16]:
df_enfermedades_select=selection_and_preparation_data(df_enfermedades_cleaned)

In [17]:
print ("Después de filtrado por sintomas filas")
print ("Enfermedades: ", df_enfermedades_select["Enfermedad"].nunique())
print ("Sintomas: ", df_enfermedades_select["Sintoma"].nunique())
print ("Frecuencias: ", df_enfermedades_select["Frecuencia"].nunique())  

Después de filtrado por sintomas filas
Enfermedades:  3728
Sintomas:  196
Frecuencias:  2


In [33]:
df=df_enfermedades_select.groupby (["Sintoma","Frecuencia"]).count().reset_index()
df

,Sintoma,Frecuencia,Enfermedad,Id_Enfermedad
0,Abdominal pain,Frecuente (79-30%),88,88
1,Abdominal pain,Muy frecuente (99-80%),52,52
2,Abnormal blistering of the skin,Frecuente (79-30%),28,28
3,Abnormal blistering of the skin,Muy frecuente (99-80%),47,47
4,Abnormal facial shape,Frecuente (79-30%),87,87
...,...,...,...,...
387,Weight loss,Muy frecuente (99-80%),42,42
388,Wide mouth,Frecuente (79-30%),37,37
389,Wide mouth,Muy frecuente (99-80%),31,31
390,Wide nasal bridge,Frecuente (79-30%),100,100


In [43]:
pd.pivot_table(df, values=["Sintoma","Frecuencia"], index=["Sintoma"])

ValueError: Grouper for 'Sintoma' not 1-dimensional

In [21]:
df_enfermedades_select.head(10)

,Enfermedad,Id_Enfermedad,Sintoma,Frecuencia
0,Enfermedad de Alexander,0,Macrocephaly,Muy frecuente (99-80%)
1,Enfermedad de Alexander,0,Intellectual disability,Muy frecuente (99-80%)
2,Enfermedad de Alexander,0,Seizure,Muy frecuente (99-80%)
3,Enfermedad de Alexander,0,Spasticity,Muy frecuente (99-80%)
4,Enfermedad de Alexander,0,Agenesis of corpus callosum,Muy frecuente (99-80%)
5,Enfermedad de Alexander,0,Hyperreflexia,Muy frecuente (99-80%)
7,Enfermedad de Alexander,0,Failure to thrive,Muy frecuente (99-80%)
8,Enfermedad de Alexander,0,Frontal bossing,Muy frecuente (99-80%)
9,Enfermedad de Alexander,0,Nausea and vomiting,Muy frecuente (99-80%)
10,Enfermedad de Alexander,0,Neurological speech impairment,Muy frecuente (99-80%)


In [22]:
df_enfermedades_select=df_enfermedades_select.drop("Id_Enfermedad", axis=1)
df_enfermedades_select

,Enfermedad,Sintoma,Frecuencia
0,Enfermedad de Alexander,Macrocephaly,Muy frecuente (99-80%)
1,Enfermedad de Alexander,Intellectual disability,Muy frecuente (99-80%)
2,Enfermedad de Alexander,Seizure,Muy frecuente (99-80%)
3,Enfermedad de Alexander,Spasticity,Muy frecuente (99-80%)
4,Enfermedad de Alexander,Agenesis of corpus callosum,Muy frecuente (99-80%)
...,...,...,...
112648,Artrogriposis múltiple congénita miogénica aut...,Hyporeflexia,Muy frecuente (99-80%)
112649,Artrogriposis múltiple congénita miogénica aut...,Motor delay,Muy frecuente (99-80%)
112652,Artrogriposis múltiple congénita miogénica aut...,Inability to walk,Muy frecuente (99-80%)
112657,Artrogriposis múltiple congénita miogénica aut...,Cryptorchidism,Frecuente (79-30%)


In [23]:
%%time
df_train_generated=generate_data_train (df_enfermedades_select, 10)

VUELTA:  1
Tamaño:  3728
VUELTA:  2
Tamaño:  7456
VUELTA:  3
Tamaño:  11184
VUELTA:  4
Tamaño:  14912
VUELTA:  5
Tamaño:  18640
VUELTA:  6
Tamaño:  22368
VUELTA:  7
Tamaño:  26096
VUELTA:  8
Tamaño:  29824
VUELTA:  9
Tamaño:  33552
VUELTA:  10
Tamaño:  37280
CPU times: user 1min 49s, sys: 40.5 s, total: 2min 30s
Wall time: 2min 30s


In [32]:
df_train_generated.shape

(37280, 197)

In [33]:
df_train_generated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37280 entries, 0 to 37279
Columns: 197 entries, id_Enfermedad to Inability to walk
dtypes: int64(197)
memory usage: 57.3 MB


In [25]:
save_to_csv(df_train_generated,"../data/01_raw/df_trained_Ofi.csv")

In [4]:
df_train_generated=pd.read_csv("../data/01_raw/df_trained_Ofi.csv")

In [26]:
df_train_generated

Sintoma,id_Enfermedad,Macrocephaly,Intellectual disability,Seizure,Spasticity,Agenesis of corpus callosum,Hyperreflexia,Failure to thrive,Frontal bossing,Nausea and vomiting,...,Generalized hypotonia,Dementia,Lymphadenopathy,Hypoplasia of the corpus callosum,Gait ataxia,Small for gestational age,Infantile muscular hypotonia,Aggressive behavior,Babinski sign,Inability to walk
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37275,3724,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
37276,3725,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
37277,3726,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37278,3727,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
df_train_generated,model,target,X_test,y_test, X_train, y_train=train_data (df_train_generated)  

In [ ]:
evaluate (model, X_test, y_test)

In [ ]:
import pickle

#pickle.dump(model,open("modelo19m.pickle","wb"))
model2=pickle.load(open("modelo19m.pickle","rb"))

In [ ]:
df_train_generated.to_csv("df_train_final.csv", index=False)

In [ ]:
from joblib import dump
dump(model, 'modelo19m.joblib') 

In [ ]:
from joblib import load
new_model = load('modelo19m.joblib')

In [ ]:
evaluate (new_model, X_test, y_test)

In [ ]:
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

In [ ]:
CV_rfc = GridSearchCV(estimator=model, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)

In [ ]:
CV_rfc.best_params_

In [ ]:
X_test

In [ ]:
X_test.iloc[[1]]

In [ ]:
model.predict(X_test.iloc[[2]])

In [ ]:
y_test.iloc[[2]]

In [ ]:
predicciones = model.predict_proba(X_test.iloc[[2]])
predicciones.shape


In [ ]:
X_test.iloc[[2]]


In [ ]:
a=df_train_generated.columns


In [ ]:
b=a.to_list()
b.remove('id_Enfermedad')
b

In [ ]:
df_train_generated.columns
df_predicciones = pd.DataFrame(data=predicciones)
#df_predicciones['clasificacion_default_0.5'] = np.where(df_predicciones['0'] > df_predicciones['1'], 0, 1)
df_predicciones

In [ ]:
df_predicciones2=df_predicciones.transpose()
df_predicciones2

In [ ]:
list_to_rank=df_predicciones2[df_predicciones2[0]>0]

In [ ]:
rank = list_to_rank.sort_values(0,ascending=False)
rank

In [ ]:
rank2=rank.reset_index()

In [ ]:
rank2=rank2.head(5)
rank2

In [ ]:
rank2.merge (data)

In [ ]:
rank2["Enfermedad"]= rank2.apply(lambda row:
                                 data[data['Id_Enfermedad']==row["index"]].iloc[0][0]
                                 if data['Id_Enfermedad']==row["index"]
                                 else 0,
                                 axis=1
                                )

In [ ]:
data.iloc[0][0]

In [ ]:
for i in range(1):
    
    a=data[data['Id_Enfermedad']==rank2["index"][i]].iloc[0][0]
    a

In [ ]:
a

In [ ]:
a.iloc[0][0]

In [ ]:
list_rank=rank.to_numpy().tolist()
list_rank


In [ ]:
for i in range(5):
    print (i)
    print (lis)
        

In [ ]:
data[data['Id_Enfermedad']==2389]

#### 